# Computing lower bounds $I_w$, upper bounds $I^w$, $\Omega(I)$, and $\Omega^*(I)$

*Authors: George Balla, Daniel Corey, Igor Makhlin, and Victoria Schleis*

In this notebook, we give general methods how to compute the lower bound ideal $I_w$, the upper bound ideal $I^w$, as well as the fans $\Omega(I)$ and $\Omega^*(I)$. We then demonstrate these methods on several examples, mostly focused on several Plücker ideals.

Our code uses the package Oscar in version 1.3.1.

In [2]:
using Oscar;
using Combinatorics;
pm = Polymake;

┌ Warning: The active manifest file has dependencies that were resolved with a different julia version (1.8.4). Unexpected behavior may occur.
└ @ ~/Manifest.toml:0
┌ Error: curl_easy_setopt: 48
│ 
│ You may be using an old system libcurl library that doesn't understand options that Julia uses. You can try the following Julia code to see which libcurl library you are using:
│ 
│     using Libdl
│     filter!(contains("curl"), dllist())
│ 
│ If this indicates that Julia is not using the libcurl library that is shipped with Julia, then that is likely to be the problem. This either means:
│ 
│   1. You are using an unofficial Julia build which is configured to use a system libcurl library that is not recent enough; you may be able to fix this by upgrading the system libcurl. You should complain to your distro maintainers for allowing Julia to use a too-old libcurl version and consider using official Julia binaries instead.
│ 
│   2. You are overriding the library load path by setting `LD_

┌ Error: curl_easy_setopt: 48
│ 
│ You may be using an old system libcurl library that doesn't understand options that Julia uses. You can try the following Julia code to see which libcurl library you are using:
│ 
│     using Libdl
│     filter!(contains("curl"), dllist())
│ 
│ If this indicates that Julia is not using the libcurl library that is shipped with Julia, then that is likely to be the problem. This either means:
│ 
│   1. You are using an unofficial Julia build which is configured to use a system libcurl library that is not recent enough; you may be able to fix this by upgrading the system libcurl. You should complain to your distro maintainers for allowing Julia to use a too-old libcurl version and consider using official Julia binaries instead.
│ 
│   2. You are overriding the library load path by setting `LD_LIBRARY_PATH`, in which case you are in advanced usage territory. You can try upgrading the system libcurl, unsetting `LD_LIBRARY_PATH`, or otherwise arranging for J

Precompiling project...
   1358.3 ms  ✓ DiffResults
   1453.7 ms  ✓ NaNMath
   1611.1 ms  ✓ Adapt
   1623.9 ms  ✓ BinaryWrappers
   1843.9 ms  ✓ GAP_lib_jll
   1928.0 ms  ✓ OpenSSL_jll
   1999.8 ms  ✓ InverseFunctions
   1964.2 ms  ✓ Ninja_jll
   2165.5 ms  ✓ CommonSubexpressions
   3405.1 ms  ✓ IrrationalConstants
   1927.5 ms  ✓ Ncurses_jll
   1945.7 ms  ✓ DecFP_jll
   1597.9 ms  ✓ LLVMOpenMP_jll
   1964.7 ms  ✓ ASL_jll
   2140.0 ms  ✓ Zstd_jll
   2017.6 ms  ✓ Bzip2_jll
   2031.4 ms  ✓ GLPK_jll
   2252.2 ms  ✓ libcxxwrap_julia_jll
   2164.8 ms  ✓ LZO_jll
   4455.9 ms  ✓ ChainRulesCore
   1787.5 ms  ✓ OpenSpecFun_jll
   1875.1 ms  ✓ cohomCalg_jll
   5198.5 ms  ✓ boost_jll
   1878.0 ms  ✓ lrslib_jll
   5465.9 ms  ✓ RecipesBase
   1814.0 ms  ✓ cddlib_jll
   2710.2 ms  ✓ METIS_jll
   2301.3 ms  ✓ PPL_jll
   2364.2 ms  ✓ bliss_jll
   2241.4 ms  ✓ Readline_jll
   1882.0 ms  ✓ snappy_jll
   3634.5 ms  ✓ ChangesOfVariables
   2443.6 ms  ✓ Singular_jll
   3194.0 ms  ✓ lib4ti2_jll
   2407.6 ms

LoadError: Failed to precompile Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13] to "/home/victoria/.julia/compiled/v1.11/Oscar/jl_5anlNZ".

## Auxiliary functions: Subsets ordered in lex and revlex

Below are some functions sorting subsets with respect to the lexicographic and reverse lexicographic ordering, which we need for the preceeding code.

---

**Function:** `subsets_lex(S, k)`

*Description*: Given a *sorted* list of integers `S` and a nonnegative integer `k`, returns the subsets of `S` that have size `k` sorted in lex. 

*Example*: `subsets_lex(Vector(1:4), 2)` returns

```[[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]```

Note that `[1, 4]` appears before `[2, 3]`. 

---

**Function:** `subsets_revlex(S, k)`

*Description*: Given a *sorted* list of integers `S` and a nonnegative integer `k`, returns the subsets of `S` that have size `k` sorted in *revlex*. 

*Example*: `subsets_revlex(Vector(1:4), 2)` returns

```[[1, 2], [1, 3], [2, 3], [1, 4], [2, 4], [3, 4]]```

Note that `[2, 3]` appears before `[1, 4]`. 



In [3]:
function subsets_lex(S::Vector{Int64}, k::Int64)
    return sort!(subsets(S,k))
end

function subsets_revlex(S::Vector{Int64}, k::Int64)
    Sk = sort!([reverse!(a) for a in subsets(S,k)])
    return [reverse!(a) for a in Sk]
end

vector_to_string (generic function with 1 method)

## Point configurations from lineality spaces

In this section, we automate the computation of the point configuration associated to an ideal. Our algorithmic process is outlined in Appendix B, and mainly relies on computing the lineality space of an ideal first.


---

**Function:** `lineality_space_H_rep(I)`

*Description*: Given an ideal `I` of a polynomial ring, returns the lineality space of the associated point configuration $\mathcal{A}(I)$ of `I` in its hyperplane representation. 

---

**Function:** `lineality_space_V_rep(I)`

*Description*: Given an ideal `I` of a polynomial ring, returns the lineality space of the associated point configuration $\mathcal{A}(I)$ of `I`, in its vertex representation. 

In [ ]:
function lineality_space_H_rep(I)
    Grob = groebner_basis(I, complete_reduction = true)
    W_perp = Vector{Vector{Int64}}()
    for f in Grob
        exponents_f = collect(exponents(f))
        for i in 1:length(exponents_f) - 1, j in i+1:length(exponents_f)
            push!(W_perp, exponents_f[i] - exponents_f[j])
        end
    end
    n, W_perp = rref(matrix(QQ, W_perp))
    return W_perp[1:n,:]
end

function lineality_space_V_rep(I)
    H_rep = lineality_space_H_rep(I)
    V_rep = transpose(kernel(H_rep, side = :right))
    n, V_rep = rref(V_rep)
    return V_rep
end

From the lineality space, we now compute the point configuration. 

---

**Function:** `point_configuration(I)`

*Description*: Given an ideal `I` of a polynomial ring, returns the associated point configuration $\mathcal{A}(I)$ of `I`. 

In [ ]:
function point_configuration(I)
    M = lineality_space_V_rep(I)
    A = [M[:,i] for i in (1:ncols(M))]
    return A
end

*Example*: We demonstrate the computation on the Plücker ideal $$I_{2,4} = \langle x_{12}x_{34} - x_{13}x_{24} + x_{14}x_{23} \rangle 
\subset \mathbb{Q}[x_{12}, x_{13}, x_{14}, x_{23}, x_{24}, x_{34}].$$ Observe that the point configuration we obtain is not the hypersimplex one usually associates to $I_{2,4}$, but instead an affinely equivalent point configuration.

In [ ]:
Plu, x = polynomial_ring(QQ, "x" => subsets_lex([1,2,3,4], 2));
I24 = grassmann_pluecker_ideal(Plu, 2, 4);
point_configuration(I24)

## Intersect with coordinate strata

Let $S$ be a polynomial ring, say $S = \mathbb{Q}[x_1,\ldots,x_n]$, and $I$ an ideal of $S$. Given a collection of coordinates $B \subset \{x_1,\ldots,x_n\}$, we form the ideal:

$$I_{B} = (I + \langle x_i \, : \, x_i \notin B \rangle ) \cap \mathbb{Q}[x_{i} \, : \, x_{i} \in B]$$

We compute the ideal $\tilde{I}_B$, which is generated by $I_B$ in $S$. We can achieve this with the following function.

**Function:** `stratum(I, B)`

*Description*: Given an ideal `I` of a polynomial ring and `B` a subset of the generators of $S$, return the ideal $\tilde{I}_{B}$ above. 


In [ ]:
function stratum(I, B)
    R = base_ring(I)
    x = gens(R)
    xNB = [x[i] for i in 1:length(x) if !(x[i] in B)]
    return eliminate(I + ideal(R, xNB), xNB)
end

*Example*: Consider the Plücker ideal $I_{2,4}$
and $B = \{x_{12}, x_{13}, x_{14}, x_{23}, x_{24}\}$. Then $I_{B} =  \langle - x_{13}x_{24} + x_{14}x_{23} \rangle$.


In [ ]:
B_no_34 = [x[i] for i in 1:5]
stratum(I24, B_no_34) 

## Subdivisions, ideals of strata, $I_w$, and $I^w$

Now suppose $\Delta = \mathcal{A}(I)$ is the point configuration of $I$ from the paper, in particular the points are labeled by the coordinates in the polynomial ring $S$. A vector $\mathsf{w}\in \mathbb{R}^{n}$ induces a regular subdivision $\mathrm{subd}_{\mathsf{w}}(\Delta)$ of $\Delta$. We can form the ideals 

$$\{\tilde{I}_{B} \, : \, B \text{ is a maximal cell of } \mathrm{subd}_{\mathsf{w}}(\Delta)\}$$

and the ideal they generate in $S$ is

$$I_{\mathsf{w}} = \sum_{B \text{ maximal}} \tilde{I}_{B}. $$

---

**Function:** `ideals_of_max_cells(I, w, Delta=point_configuration(I))`

*Description*: Given an ideal `I` of a polynomial ring, `w` a vector of length equal to the number of generators of $S$, and `Delta` the point configuration of `I`, return a lists of the ideals $\tilde{I}_{B}$ above, as $B$ runs through the coordinates of the maximal cells. The input of `Delta` is optional, if no point configuration is provided, the point configuration is computed by the function `point_configuration(I)` provided above.

---

**Function:** `ideal_w(I, w, Delta=point_configuration(I))`

*Description*: Given an ideal `I`,  `w`, and an optional point configuration `Delta` as above, returns the ideal $I_{\mathsf{w}}$.

In [1]:
function ideals_of_max_cells(I, w, Delta=point_configuration(I))
    x = gens(base_ring(I))
    subd = subdivision_of_points(Delta, w)
    return [stratum(I, x[B]) for B in maximal_cells(subd)]
end

function ideal_w(I, w, Delta=point_configuration(I))
    return sum(ideals_of_max_cells(I, w, Delta))
end

ideal_w (generic function with 2 methods)

*Example*: Consider the Plücker ideal $I_{2,4}$ as in the examples above. 

In [ ]:
w = [1,0,0,0,0,0]
ideals_of_max_cells(I24, w) 
ideal_w(I24, w)

Note that the vertices line up in order with the order of the Plücker coordinates in the stratum example. 

That is, $I_{\mathsf{w}} = \langle - x_{13}x_{24} + x_{14}x_{23} \rangle$.

Similarly, we can compute the ideal 

$$I^w = \bigcap_{B\text{ cell of }\Delta^*} \tilde{I}^B$$

as the intersection of the ideals $\tilde{I}_B$ associated to maximal cells.  

---

**Function:** `ideal_up_w(I, w, Delta=point_configuration(I))`

*Description*: Given an ideal `I`, `w`, and an optional point configuration `Delta` as above, returns the ideal $I^{\mathsf{w}}$.

In [ ]:
function ideal_up_w(I, w, Delta=point_configuration(I))
    R = base_ring(I)
    x = gens(R)
    subd = subdivision_of_points(Delta, w)
    Iw=ideal(R, [R(1)])
    for C in maximal_cells(subd)
        NV = [gens(R)[i] for i in setdiff(1:length(Delta), C)]
        SC, x = polynomial_ring(QQ, "x" => sort(C))
        f = hom(SC, R, [gens(R)[i] for i in sort(C)])
        CI = preimage(f, I)
        G = ([R(f(g)) for g in gens(CI)])
        append!(G,NV)
        Iw_new = ideal(R,G)
        Iw = intersect(Iw,ideal(R,G))
    end
    return Iw
end

*Example:* For a random point configuration `Delta`, we compute the associated toric ideal `I` and the ideal $I^w$.

In [ ]:
d = 3;
n = 6;

R, x = graded_polynomial_ring(QQ, "x" => (1:n));
S, t, z = polynomial_ring(QQ, [:t], "z" => (1:d));
t = t[1];

A = [rand(0:10, d) for _ in (1:n)];
f = hom(R, S, [t*prod(z.^A[i]) for i in (1:n)]);
I = kernel(f);

w = rand(0:1, n);

Iw = ideal_up_w(I, w, A)

## Initial ideals

Oscar has a built-in function for handling initial ideals. For example,
```
nu_t = tropical_semiring_map(QQ, min)
initial(I24, nu_t, [1,0,0,0,0,0]) == ideal(Plu, -x[2]*x[5] + x[3]*x[4])
```
computes the initial ideal $\mathrm{in}_{\mathsf{w}} I_{2,4} = \langle - x_{13}x_{24} + x_{14}x_{23} \rangle$. 

In [ ]:
nu_t = tropical_semiring_map(QQ, min);

## The fan $\Omega(I)$

Recall that, for an ideal $I$ with point configuration $\Delta$, we define $\Omega(I)$ by

$$ \Omega(I) = \{\mathsf{w} \in \mathbb{R}^{n} \, : \, \mathrm{in}_{\mathsf{w}} I = I_{\mathsf{w}} \}. $$

This is a subfan of the secondary fan $\mathrm{Sec}(\Delta)$. 

---

**Function:** `Omega_fan(I, Delta, Sec, outside = false)`

*Description*: Given an ideal `I` of a polynomial ring, `Delta` the point configuration of `I`, and `Sec` the secondary fan of `Delta`, returns the fan $\Omega(I)$ as a pair of rays and cones. If the option `outside = true`, then this lists the cones of `Sec` that are *not* in `Omega`. 

Here, `Sec` can be given as a `PolyhedralFan` or as a pair `[rays_Delta, cones_Delta]` where `rays_Delta` is a `Vector{RayVector}` containing the rays and `cones_Delta` is a `IncidenceMatrix` recording the cones of `Sec`. The latter is useful when `Sec` is computed up to symmetry. 

In [ ]:
function Omega_fan(I, Delta, Sec, outside = false)

    if typeof(Sec) == PolyhedralFan{QQFieldElem}
        rays_Delta = rays_modulo_lineality(Sec)[1];
        cones_Delta = cones(Sec);
    else
        rays_Delta, cones_Delta = Sec
    end

    n_cones_Delta, n_rays_Delta = size(cones_Delta)
    reps = [sum([rays_Delta[j] for j in 1:n_rays_Delta if cones_Delta[i,j]]) 
                               for i in 1:n_cones_Delta];

    tests = []
    for w in reps
        w = Vector{Int64}(pm.common.primitive(w))
        init_w_I = initial(I, nu_t, w)
        I_w = ideal_w(I,  w, vertices(Delta))
        push!(tests, init_w_I == I_w)
    end
    
    if outside
        outside_Omega = [i for i in 1:length(reps) if !tests[i]]
        return [rays_Delta, cones_Delta[outside_Omega, :]]
    else
        inside_Omega = [i for i in 1:length(reps) if tests[i]];
        return [rays_Delta, cones_Delta[inside_Omega, :]]
    end
    
end

*Example*: We see that $\Omega(I_{2,4}) = \mathrm{Sec}(\Delta(2,4))$,

In [ ]:
Plu24, x = polynomial_ring(QQ, "x" => subsets_lex([1,2,3,4], 2));
I24 = grassmann_pluecker_ideal(Plu24, 2, 4);
Delta24 = hypersimplex(2,4);
Delta24_polymake = Polymake.polytope.hypersimplex(2,4);
Sec24 = polyhedral_fan(Polymake.fan.secondary_fan(Delta24_polymake));
Omega_fan(I24, Delta24, Sec24)

but there are 45 cones of  $\mathrm{Sec}(\Delta(2,5))$ not contained in $\Omega(I_{2,5})$, and these fall into two $\mathfrak{S}_{5}$ symmetry classes: 30 of them are maximal while 15 have codimension 1.

In [ ]:
Plu25, x = polynomial_ring(QQ, "x" => subsets_lex([1,2,3,4,5], 2));
I25 = grassmann_pluecker_ideal(Plu25, 2, 5);
Delta25 = hypersimplex(2,5);

Delta25_polymake = Polymake.polytope.hypersimplex(2,5);
Sec25 = polyhedral_fan(Polymake.fan.secondary_fan(Delta25_polymake));
Omega25 = Omega_fan(I25, Delta25, Sec25, true);
Omega25[2]

## Comparing Tropicalization and $\Omega(I)$

We now give the computations for Remark 5.5. Here, we compute the intersection of the tropicalization of the Grassmannian $\mathrm{TGr}(3,6)$ with the fan $\Omega(I)$. Since the secondary fan is very large, instead of computing it upfront and then intersecting with the tropical Grassmannian, we start with the tropical Grassmannian and check for each cone whether it is contained in $\Omega(I)$. To simplify computations, we load a file containing the pre-computed $\mathrm{TGr}(3,6)$. This file has been computed by Bendle, Böhm, Ren, and Schröter and can be downloaded <a href="https://github.com/micjoswig/oscar-notebooks/blob/master/Tropical%2BGrassmannian/TGr36.mrdi">here</a>. It is a result of the <a href="https://www.mardi4nfdi.de/about/mission">MaRDI</a>-project, in a format developed by Della Vecchia, Joswig and Lorenz. Warning: This computation takes a few minutes.

In [ ]:
k=3; n=6

PI = grassmann_pluecker_ideal(k, n);
Deltakn = vertices(hypersimplex(k,n));
S = base_ring(PI);
T = load("TGr36.mrdi");
R = rays_modulo_lineality(T)[1];
C = cones(T);
F = [];
notF = [];
for i in (1:n_rows(C))
    w = sum([C[i,j]*R[j] for j in (1:length(R))])
    w = -lcm(denominator.(w))*w
    c = cone([C[i,j]*R[j] for j in (1:length(R))])

    inI = initial(PI, tropical_semiring_map(QQ), w)
    Iw = ideal_w(PI,w,Deltakn)
    if (Iw == inI)
        push!(F, c)
    else
        push!(notF, c)
    end
end
(F,notF)

We can count the number of cones of specific dimension lying in the intersection, using the following.

In [ ]:
using DataStructures;
counter(dim.(F))

## The fan $\Omega^*(I)$

Recall that, for an ideal $I$ with point configuration $\Delta$, we define $\Omega^*(I)$ similar to $\Omega(I)$ by

$$ \Omega^*(I) = \{\mathsf{w} \in \mathbb{R}^{n} \, : \, \mathrm{in}_{\mathsf{w}} I = I^{\mathsf{w}} \}. $$

This is a subfan of the negative secondary fan $-\mathrm{Sec}(\Delta)$. 

---

**Function:** `Omega_star_fan(I, Delta, Sec, outside = false)`

*Description*: Given an ideal `I` of a polynomial ring, `Delta` the point configuration of `I`, and `Sec` the secondary fan of `Delta`, returns the fan $\Omega(I)$ as a pair of rays and cones. If the option `outside = true`, then this lists the cones of `-Sec` that are *not* in `Omega`. 

Here, `Sec` can be given as a `PolyhedralFan` or as a pair `[rays_Delta, cones_Delta]` where `rays_Delta` is a `Vector{RayVector}` containing the rays and `cones_Delta` is a `IncidenceMatrix` recording the cones of `Sec`. The latter is useful when `Sec` is computed up to symmetry. 

In [ ]:
function Omega_star_fan(I, Delta, Sec, outside = false)

    if typeof(Sec) == PolyhedralFan{QQFieldElem}
        rays_Delta = rays_modulo_lineality(Sec)[1];
        cones_Delta = cones(Sec);
    else
        rays_Delta, cones_Delta = Sec
    end

    n_cones_Delta, n_rays_Delta = size(cones_Delta)
    reps = [sum([rays_Delta[j] for j in 1:n_rays_Delta if cones_Delta[i,j]]) 
                               for i in 1:n_cones_Delta];

    tests = []
    for w in reps
        w = Vector{Int64}(pm.common.primitive(w))
        init_w_I = initial(I, nu_t, -w)
        I_w = ideal_up_w(I, w, Delta)
        push!(tests, init_w_I == I_w)
    end
    
    if outside
        outside_Omega = [i for i in 1:length(reps) if !tests[i]]
        return [rays_Delta, cones_Delta[outside_Omega, :]]
    else
        inside_Omega = [i for i in 1:length(reps) if tests[i]];
        return [rays_Delta, cones_Delta[inside_Omega, :]]
    end
    
end

*Example*: We compute the fan $\Omega^*(I_{2,4})$ for the ideal $I_{2,4}$ considered in the previous examples.

In [ ]:
Omega_star_fan(I24, Delta24, Sec24)

For the toric ideal `I` associated to the 3-cube, we compute the fan $\Omega^{*}(I)$ as follows. Warning: This computation can go wrong if the 

In [ ]:
d = 3;
n = 8;

R, x = graded_polynomial_ring(QQ, "x" => (1:n));
S, t, z = polynomial_ring(QQ, [:t], "z" => (1:d));
t = t[1];
PDelta = cube(3,0,1);
Delta = vertices(PDelta);
f = hom(R, S, [t*prod(z.^Int.(Delta[i])) for i in (1:n)]);
I = kernel(f);
Sec = polyhedral_fan(Polymake.fan.secondary_fan(PDelta.pm_polytope);

Omega_star_fan(I, Delta, Sec)